In [1]:
import cv2 as cv
import numpy as np

inputMode = True
roiPts = [] 
frame = None
roiBox = None

# callback del mouse
def selectROI(event, x, y, flags, param):
    global frame, roiPts, inputMode
    if inputMode and event==cv.EVENT_LBUTTONDOWN and len(roiPts) < 4:
        roiPts.append((x, y))
        # pone el el frame el punto seleccionado
        cv.circle(frame, (x, y), 4, (0, 255, 0), 2)
        
    
cam = cv.VideoCapture(0)
term_crit = ( cv.TERM_CRITERIA_COUNT | cv.TERM_CRITERIA_EPS, 10, 1 )
cv.namedWindow('Cam')  
cv.setMouseCallback('Cam',selectROI)  

while True:
   
    ret, frame = cam.read()

    if ret == True:
        
        # espera en input mode
        
        while len(roiPts) < 4:
            cv.imshow("Cam", frame)
            cv.waitKey(0)
               
                
        
        # determine the top-left and bottom-right points
        roiPts = np.array(roiPts)
        s = roiPts.sum(axis = 1)
        tl = roiPts[np.argmin(s)]
        br = roiPts[np.argmax(s)]

        # grab the ROI for the bounding box and convert it to the HSV color space
        roi = orig[tl[1]:br[1], tl[0]:br[0]]
        roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)

        # compute a HSV histogram for the ROI and store the bounding box
        roiHist = cv.calcHist([roi], [0], None, [16], [0, 180])
        roiHist = cv.normalize(roiHist, roiHist, 0, 255, cv.NORM_MINMAX)
        return (tl[0], tl[1], br[0], br[1])
            
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0],roiHist,[0,180],1)
        r, roiBox = cv.CamShift(backProj, roiBox, termination)
        pts = np.int0(cv.cv.BoxPoints(r))
        img = cv.polylines(frame, [pts], True, (0, 255, 0), 2)
        cv.imshow('Cam',img)
        
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break

        
    else:
        break

cv.destroyAllWindows()
cam.release()

SyntaxError: 'return' outside function (<ipython-input-1-f974d4f1c42d>, line 76)